In [1]:
import os
import time
from contextlib import contextmanager
from typing import Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    f1_score,
    mean_absolute_error,
    mean_squared_error,
)

import torch
from torch import Tensor


from tqdm import tqdm  # Add tqdm for progress bar
from itertools import product

# RelBench
from relbench.datasets import get_dataset
from relbench.tasks import get_task

import relbench.metrics
import inspect
from skrub import TableVectorizer

# TabPFN
from tabpfn import TabPFNClassifier, TabPFNRegressor

from relbench.modeling.utils import get_stype_proposal
from relbench.modeling.graph import make_pkey_fkey_graph, get_node_train_table_input
from torch_geometric.loader import NeighborLoader
from torch_frame.config.text_embedder import TextEmbedderConfig
from sentence_transformers import SentenceTransformer

# --- ADD: Import Model, test, BCEWithLogitsLoss, L1Loss from train_model.ipynb ---t
from torch.nn import BCEWithLogitsLoss, L1Loss
# --- END ADD ---

from typing import List, Optional


# Device preference
#if torch.backends.mps.is_available():
#    DEVICE = "mps"
#elif torch.cuda.is_available():
#    DEVICE = "cuda"
#else:
#    DEVICE = "cpu"

DEVICE = "cpu"

print(f"Using device: {DEVICE}")

# Define global dataset variable
#DATASET = "rel-f1"


Using device: cpu


In [2]:
# --- Dataset/task discovery (dataset-agnostic) ---
from relbench.datasets import get_dataset
from relbench.tasks import get_task, get_task_names
from relbench.base import TaskType

# Reuse existing config if present, otherwise set defaults
DATASET = globals().get("DATASET", "rel-f1")
DOWNLOAD = globals().get("DOWNLOAD", True)

# Discover tasks and keep only entity-level cls/reg tasks TabPFN can handle
def _is_tabpfn_friendly(task):
    return task.task_type in (
        TaskType.BINARY_CLASSIFICATION,
        TaskType.MULTICLASS_CLASSIFICATION,
        TaskType.MULTILABEL_CLASSIFICATION,
        TaskType.REGRESSION,
    )

_all = get_task_names(DATASET)  # shown in tutorials
TASKS = []
for tname in _all:
    try:
        t = get_task(DATASET, tname, download=DOWNLOAD)
        if _is_tabpfn_friendly(t):
            TASKS.append(tname)
    except Exception as e:
        print(f"[skip] {tname}: {e!s}")

print(f"{DATASET}: {len(TASKS)} TabPFN-friendly tasks -> {TASKS}")


rel-f1: 3 TabPFN-friendly tasks -> ['driver-position', 'driver-dnf', 'driver-top3']


In [3]:
# Patch relbench.metrics.skm.mean_squared_error to local mean_squared_error
relbench.metrics.skm.mean_squared_error = mean_squared_error

def patched_rmse(true, pred):
    if "squared" in inspect.signature(mean_squared_error).parameters:
        return mean_squared_error(true, pred, squared=False)
    else:
        return np.sqrt(mean_squared_error(true, pred))

relbench.metrics.rmse = patched_rmse

In [4]:
class GloveTextEmbedding:
    def __init__(self, device: Optional[torch.device] = None):
        self.model = SentenceTransformer(
            "sentence-transformers/average_word_embeddings_glove.6B.300d",
            device=device,
        )

    def __call__(self, sentences: List[str]) -> Tensor:
        return torch.from_numpy(self.model.encode(sentences))

In [5]:
@contextmanager
def elapsed_timer():
    start = time.perf_counter()
    yield lambda: time.perf_counter() - start

def classification_metrics(y_true, y_pred, y_prob=None) -> Dict[str, float]:
    out = {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
    }
    if y_prob is not None:
        try:
            out["roc_auc"] = roc_auc_score(y_true, y_prob)
        except Exception:
            out["roc_auc"] = np.nan
    else:
        out["roc_auc"] = np.nan
    return out

def regression_metrics(y_true, y_pred, y_prob=None) -> Dict[str, float]:
    # Accepts y_prob for compatibility, but ignores it
    return {
        "mae": mean_absolute_error(y_true, y_pred),
        "mse": mean_squared_error(y_true, y_pred),
    }

dataset = get_dataset(DATASET)
db = dataset.get_db()

def to_pandas(table):
    if hasattr(table, "to_pandas"):
        return table.to_pandas()
    if hasattr(table, "df"):
        return table.df
    raise ValueError("Unknown table type")

# Convert all tables to pandas DataFrames
tables = {}
for name in db.table_dict:
    tables[name] = to_pandas(db.table_dict[name])

for df in tables.values():
    for col in df.columns:
        if "date" in col.lower():
            # 1) Standardize and coerce to UTC
            df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)
            # 2) Fill missing with earliest valid timestamp
            df[col] = df[col].fillna(df[col].min())
            # 3) Extract calendar components
            df[f"{col}_year"]    = df[col].dt.year
            df[f"{col}_month"]   = df[col].dt.month
            df[f"{col}_day"]     = df[col].dt.day
            df[f"{col}_weekday"] = df[col].dt.weekday
            df[f"{col}_quarter"] = df[col].dt.quarter
            # 4) Binary indicators for edges of period
            df[f"{col}_is_month_start"] = df[col].dt.is_month_start.astype(int)
            df[f"{col}_is_month_end"]   = df[col].dt.is_month_end.astype(int)
            df[f"{col}_is_weekend"]     = (df[col].dt.weekday >= 5).astype(int)
            # 5) Cyclical encodings to preserve periodicity
            df[f"{col}_month_sin"]   = np.sin(2 * np.pi * df[f"{col}_month"]   / 12)
            df[f"{col}_month_cos"]   = np.cos(2 * np.pi * df[f"{col}_month"]   / 12)
            df[f"{col}_weekday_sin"] = np.sin(2 * np.pi * df[f"{col}_weekday"] /  7)
            df[f"{col}_weekday_cos"] = np.cos(2 * np.pi * df[f"{col}_weekday"] /  7)
            # 6) Relative elapsed days since earliest date
            df[f"{col}_elapsed_days"] = (df[col] - df[col].min()).dt.days
            # Strip timezone so to_unix_time sees datetime64[ns]
            df[col] = df[col].dt.tz_localize(None)
# --- ADD: push processed tables back into db.table_dict ---
for name, df in tables.items():
    db.table_dict[name].df = df
# --- END ADD ---

# --- ADD THIS BLOCK: Build the hetero-temporal graph for GNN experiments ---
col_to_stype_dict = get_stype_proposal(db)
text_embedder_cfg = TextEmbedderConfig(
    text_embedder=GloveTextEmbedding(device=torch.device(DEVICE)), batch_size=256
)
data, col_stats_dict = make_pkey_fkey_graph(
    db,
    col_to_stype_dict=col_to_stype_dict,
    text_embedder_cfg=text_embedder_cfg,
    cache_dir=None,
)

Loading Database object from /Users/michaelflppv/Library/Caches/relbench/rel-f1/db...
Done in 0.01 seconds.


Embedding raw data in mini-batch: 100%|██████████| 4/4 [00:00<00:00, 808.89it/s]
/Users/michaelflppv/PycharmProjects/TabPFN/.venv1/lib/python3.10/site-packages/torch_frame/data/stats.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ser = pd.to_datetime(ser, format=time_format)
Embedding raw data in mini-batch: 100%|██████████| 4/4 [00:00<00:00, 603.32it/s]
/Users/michaelflppv/PycharmProjects/TabPFN/.venv1/lib/python3.10/site-packages/torch_frame/data/mapper.py:291: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ser = pd.to_datetime(ser, format=self.format, errors='coerce')
Embedding raw data in mini-batch: 100%|██████████| 1/1 [00:00<00:00, 529.12it/s]


In [6]:
# --- Split fetch + X/y builders (dataset-agnostic) ---
import pandas as pd
import numpy as np

def fetch_splits(dataset_name: str, task_name: str, download: bool = True):
    task = get_task(dataset_name, task_name, download=download)
    # keep original columns (tutorial shows mask_input_cols flag)
    splits = {
        split: task.get_table(split, mask_input_cols=False)
        for split in ("train", "val", "test")
    }
    return task, splits

def to_Xy(df: pd.DataFrame, target_col: str):
    y = df[target_col].to_numpy()
    X = df.drop(columns=[target_col])
    return X, y

# --- Naive merged table via one-hop FK→PK joins using dataset.get_db() ---
# Uses only duck-typing on Table objects; falls back gracefully if metadata is missing.
def _infer_pk(table_obj, df: pd.DataFrame):
    # best-effort: check common attribute names first, then infer by uniqueness
    for attr in ("primary_key_col", "pkey", "pk", "primary_key", "id_col"):
        if hasattr(table_obj, attr):
            cand = getattr(table_obj, attr)
            if isinstance(cand, str) and cand in df.columns:
                return cand
    # fallback: take the first unique column if exists
    for c in df.columns:
        try:
            if df[c].is_unique:
                return c
        except Exception:
            pass
    return None

def denormalize_one_hop(dataset, base_df: pd.DataFrame):
    try:
        db = dataset.get_db()  # documented in README
    except Exception:
        return base_df  # cannot access DB; return base table

    tables = getattr(db, "tables", None)
    if not isinstance(tables, dict):
        return base_df

    # Build a map: PK column name -> (table_name, table_df, pk_col)
    pkmap = {}
    for name, tbl in tables.items():
        df = getattr(tbl, "df", None)
        if isinstance(df, pd.DataFrame):
            pk = _infer_pk(tbl, df)
            if pk and pk in df.columns:
                pkmap.setdefault(pk, []).append((name, df, pk))

    df_out = base_df.copy()
    # For each column in base that matches a PK in the DB, left join the non-key attributes
    for fk_col in list(df_out.columns):
        if fk_col in pkmap:
            for (tname, tdf, pk) in pkmap[fk_col]:
                right = tdf.drop(columns=[pk], errors="ignore").copy()
                if right.empty:
                    continue
                # prefix joined columns to avoid collisions
                right = right.add_prefix(f"{tname}__")
                # attach the join key back (unprefixed) for merge
                right[fk_col] = tdf[pk]
                try:
                    df_out = df_out.merge(right, on=fk_col, how="left")
                except Exception:
                    # keep going; some keys may be non-joinable due to dtype issues
                    pass
    return df_out

def build_single_table_frames(task, splits):
    frames = {}
    for split, table in splits.items():
        df = table.df.copy()
        # IMPORTANT: do not touch column masking/order; just drop the target to form X
        X, y = to_Xy(df, task.target_col)
        frames[split] = (X, y, df)  # keep original df for evaluation alignment
    return frames

def build_merged_table_frames(dataset, task, splits):
    frames = {}
    for split, table in splits.items():
        base_df = table.df.copy()
        merged_df = denormalize_one_hop(dataset, base_df)
        X, y = to_Xy(merged_df, task.target_col)
        frames[split] = (X, y, merged_df)
    return frames


In [7]:
# --- NEW CELL: skrub preprocessing helpers ---

def build_tv():
    """
    TableVectorizer turns mixed (numeric + categorical + text + datetime) columns
    into a numeric feature matrix. We keep defaults to match the tutorials’ simplicity.
    """
    return TableVectorizer()

def fit_transform_splits(tv, X_train_df, X_val_df=None, X_test_df=None):
    """
    Fit TV on train only; transform val/test. Cast to float32 for TabPFN.
    """
    # Ensure no NaN or infinite values before vectorizing
    X_train_df = X_train_df.replace([np.inf, -np.inf], np.nan).fillna(0)
    if X_val_df is not None:
        X_val_df = X_val_df.replace([np.inf, -np.inf], np.nan).fillna(0)
    if X_test_df is not None:
        X_test_df = X_test_df.replace([np.inf, -np.inf], np.nan).fillna(0)
    Xt_train = tv.fit_transform(X_train_df).astype(np.float32)
    Xt_val   = tv.transform(X_val_df).astype(np.float32) if X_val_df is not None else None
    Xt_test  = tv.transform(X_test_df).astype(np.float32) if X_test_df is not None else None
    return Xt_train, Xt_val, Xt_test, tv

def _get_df(table):
    # Your tutorial-style accessor: works whether object has `.df` or `.to_pandas()`
    if hasattr(table, "df"):
        return table.df
    if hasattr(table, "to_pandas"):
        return table.to_pandas()
    raise ValueError("Unknown table type for conversion to DataFrame.")


In [8]:
# --- Vectorization wrapper (version-safe for skrub/dirty_cat) ---
# Place this where your previous TableVectorizer/vectorize_splits block was.

def _make_table_vectorizer():
    sig = inspect.signature(TableVectorizer.__init__)
    allowed = set(sig.parameters.keys()) - {"self"}

    tv_kwargs = {}

    # Only set kwargs that actually exist in the installed version
    if "cardinality_threshold" in allowed:
        tv_kwargs["cardinality_threshold"] = globals().get("CARDINALITY_THRESHOLD", 1000)

    # Some versions expose this; others don't — guard it
    if "high_cardinality_transformer" in allowed:
        tv_kwargs["high_cardinality_transformer"] = globals().get("HIGH_CARD_TRANSFORMER", "hashing")

    # Optional knobs if you happen to define them globally and the version supports them
    if "text_separator" in allowed and "TEXT_SEPARATOR" in globals():
        tv_kwargs["text_separator"] = globals()["TEXT_SEPARATOR"]
    if "numerical_transformer" in allowed and "NUMERICAL_TRANSFORMER" in globals():
        tv_kwargs["numerical_transformer"] = globals()["NUMERICAL_TRANSFORMER"]
    if "categorical_transformer" in allowed and "CATEGORICAL_TRANSFORMER" in globals():
        tv_kwargs["categorical_transformer"] = globals()["CATEGORICAL_TRANSFORMER"]

    return TableVectorizer(**tv_kwargs)

def _to_dense(X):
    try:
        # scipy sparse matrices have .toarray()
        return X.toarray() if hasattr(X, "toarray") else X
    except Exception:
        return X

def vectorize_splits(X_train, X_val, X_test):
    tv = _make_table_vectorizer()
    Xt = _to_dense(tv.fit_transform(X_train))
    Xv = _to_dense(tv.transform(X_val))
    Xs = _to_dense(tv.transform(X_test))
    return tv, Xt, Xv, Xs


In [9]:
# --- Generic TabPFN experiment for a given task & mode ('single' or 'merged') ---
import math
from typing import Dict, Any

try:
    from tabpfn import TabPFNClassifier, TabPFNRegressor
except Exception:
    # Older TabPFN installs may not expose the Regressor; fall back to classifier-only path
    TabPFNRegressor = None

SEED   = globals().get("SEED", 42)
DEVICE = globals().get("DEVICE", "cuda" if hasattr(np, "__cuda_array_interface__") else "cpu")
N_ENSEMBLE = globals().get("N_ENSEMBLE", 16)
TABPFN_MAX = globals().get("TABPFN_MAX", 1000)  # hard ceiling for TabPFN

def _subsample(X, y, cap=TABPFN_MAX, seed=SEED):
    if len(X) <= cap:
        return X, y, np.arange(len(X))
    idx = np.random.RandomState(seed).choice(len(X), size=cap, replace=False)
    if hasattr(X, "iloc"):
        Xs = X.iloc[idx]
    else:
        Xs = X[idx]
    ys = y[idx]
    return Xs, ys, idx

def _fit_tabpfn(task, Xt, yt):
    if task.task_type == TaskType.REGRESSION and TabPFNRegressor is not None:
        model = TabPFNRegressor(
            device=DEVICE,
            #n_estimators=int(N_ENSEMBLE),
        )
    else:
        model = TabPFNClassifier(
            device=DEVICE,
            #n_estimators=int(N_ENSEMBLE),
        )
    model.fit(Xt, yt)
    return model

def _predict_for_task(task, model, X):
    # align with RelBench evaluators: AUROC expects probabilities for the positive class
    if task.task_type == TaskType.REGRESSION:
        return model.predict(X)
    proba = model.predict_proba(X)
    if task.task_type == TaskType.BINARY_CLASSIFICATION:
        return proba[:, 1]
    else:
        # multiclass/multilabel: pass full probability matrix
        return proba

def run_tabpfn_on_task(dataset_name: str, task_name: str, mode: str = "single") -> Dict[str, Any]:
    dataset = get_dataset(dataset_name, download=DOWNLOAD)
    task, splits = fetch_splits(dataset_name, task_name, download=DOWNLOAD)

    if mode == "single":
        frames = build_single_table_frames(task, splits)
    elif mode == "merged":
        frames = build_merged_table_frames(dataset, task, splits)
    else:
        raise ValueError("mode must be 'single' or 'merged'")

    (Xtr, ytr, _dftr) = frames["train"]
    (Xva, yva, dfva)  = frames["val"]
    (Xte, yte, dfte)  = frames["test"]

    # Vectorize
    tv, Xt, Xv, Xs = vectorize_splits(Xtr, Xva, Xte)

    # Respect TabPFN's sample cap
    Xt_cap, yt_cap, _ = _subsample(Xt, ytr, cap=TABPFN_MAX, seed=SEED)

    # Fit
    model = _fit_tabpfn(task, Xt_cap, yt_cap)

    # Predict & Evaluate with RelBench evaluators
    val_pred  = _predict_for_task(task, model, Xv)
    test_pred = _predict_for_task(task, model, Xs)

    val_metrics  = task.evaluate(val_pred,  splits["val"])   # documented API
    test_metrics = task.evaluate(test_pred, splits["test"])  # documented API

    out = {
        "dataset": dataset_name,
        "task": task_name,
        "mode": mode,
        "val_metrics": val_metrics,
        "test_metrics": test_metrics,
        "n_train_used": len(Xt_cap),
        "n_train_total": len(Xt),
        "n_val": len(Xv),
        "n_test": len(Xs),
    }
    return out


In [10]:
# Create features for drivers based on their past performance
# This function creates features like average position, points, DNF rate, and average laps
def engineer_driver_features():
    # Extract the race dates and results
    results = tables["results"].merge(
        tables["races"][["raceId", "date"]],
        on="raceId", how="left"
    )

    print("results columns after merge:", results.columns.tolist())
    print("Number of rows after merge:", results.shape[0])
    results = results.dropna(subset=["driverId", "date_y"])
    print("Number of rows after dropna:", results.shape[0])

    # Create a did not finish (DNF) flag as indicator for future race outcomes
    results["dnf_flag"] = (~results["positionOrder"].isna()).astype(int)
    # For each driver, calculate the average position, points, DNF rate, and average laps
    # Only the information before the current race is used
    feats = results.groupby("driverId").expanding().agg({
        "positionOrder": "mean",
        "points": "mean",
        "dnf_flag": "mean",
        "laps": "mean"
    }).reset_index()
    feats = feats.rename(columns={
        "positionOrder": "avg_position",
        "points": "avg_points",
        "dnf_flag": "dnf_rate",
        "laps": "avg_laps"
    })
    feats["date"] = results["date_y"].values
    return feats

driver_feats = engineer_driver_features()

results columns after merge: ['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid', 'position', 'positionOrder', 'points', 'laps', 'milliseconds', 'fastestLap', 'rank', 'statusId', 'date_x', 'date_year', 'date_month', 'date_day', 'date_weekday', 'date_quarter', 'date_is_month_start', 'date_is_month_end', 'date_is_weekend', 'date_month_sin', 'date_month_cos', 'date_weekday_sin', 'date_weekday_cos', 'date_elapsed_days', 'date_y']
Number of rows after merge: 20323
Number of rows after dropna: 20323


In [11]:
from typing import Any, Dict, List

import torch
from torch import Tensor
from torch.nn import Embedding, ModuleDict
from torch_frame.data.stats import StatType
from torch_geometric.data import HeteroData
from torch_geometric.nn import MLP
from torch_geometric.typing import NodeType

from relbench.modeling.nn import HeteroEncoder, HeteroGraphSAGE, HeteroTemporalEncoder

class Model(torch.nn.Module):

    def __init__(
        self,
        data: HeteroData,
        col_stats_dict: Dict[str, Dict[str, Dict[StatType, Any]]],
        num_layers: int,
        channels: int,
        out_channels: int,
        aggr: str,
        norm: str,
        # List of node types to add shallow embeddings to input
        shallow_list: List[NodeType] = [],
        # ID awareness
        id_awareness: bool = False,
    ):
        super().__init__()

        self.encoder = HeteroEncoder(
            channels=channels,
            node_to_col_names_dict={
                node_type: data[node_type].tf.col_names_dict
                for node_type in data.node_types
            },
            node_to_col_stats=col_stats_dict,
        )
        self.temporal_encoder = HeteroTemporalEncoder(
            node_types=[
                node_type for node_type in data.node_types if "time" in data[node_type]
            ],
            channels=channels,
        )
        self.gnn = HeteroGraphSAGE(
            node_types=data.node_types,
            edge_types=data.edge_types,
            channels=channels,
            aggr=aggr,
            num_layers=num_layers,
        )
        self.head = MLP(
            channels,
            out_channels=out_channels,
            norm=norm,
            num_layers=1,
        )
        self.embedding_dict = ModuleDict(
            {
                node: Embedding(data.num_nodes_dict[node], channels)
                for node in shallow_list
            }
        )

        self.id_awareness_emb = None
        if id_awareness:
            self.id_awareness_emb = torch.nn.Embedding(1, channels)
        self.reset_parameters()

    def reset_parameters(self):
        self.encoder.reset_parameters()
        self.temporal_encoder.reset_parameters()
        self.gnn.reset_parameters()
        self.head.reset_parameters()
        for embedding in self.embedding_dict.values():
            torch.nn.init.normal_(embedding.weight, std=0.1)
        if self.id_awareness_emb is not None:
            self.id_awareness_emb.reset_parameters()

    def forward(
        self,
        batch: HeteroData,
        entity_table: NodeType,
    ) -> Tensor:
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
            batch.num_sampled_nodes_dict,
            batch.num_sampled_edges_dict,
        )

        return self.head(x_dict[entity_table][: seed_time.size(0)])

    def forward_dst_readout(
        self,
        batch: HeteroData,
        entity_table: NodeType,
        dst_table: NodeType,
    ) -> Tensor:
        if self.id_awareness_emb is None:
            raise RuntimeError(
                "id_awareness must be set True to use forward_dst_readout"
            )
        seed_time = batch[entity_table].seed_time
        x_dict = self.encoder(batch.tf_dict)
        # Add ID-awareness to the root node
        x_dict[entity_table][: seed_time.size(0)] += self.id_awareness_emb.weight

        rel_time_dict = self.temporal_encoder(
            seed_time, batch.time_dict, batch.batch_dict
        )

        for node_type, rel_time in rel_time_dict.items():
            x_dict[node_type] = x_dict[node_type] + rel_time

        for node_type, embedding in self.embedding_dict.items():
            x_dict[node_type] = x_dict[node_type] + embedding(batch[node_type].n_id)

        x_dict = self.gnn(
            x_dict,
            batch.edge_index_dict,
        )

        return self.head(x_dict[dst_table])


model = Model(
    data=data,
    col_stats_dict=col_stats_dict,
    num_layers=2,
    channels=128,
    out_channels=1,
    aggr="sum",
    norm="batch_norm",
).to(DEVICE)

In [12]:
# --- MODIFY: make test accept entity_table ---
@torch.no_grad()
def test(loader: NeighborLoader, entity_table: NodeType) -> np.ndarray:
    model.eval()
    pred_list = []
    for batch in loader:
        batch = batch.to(DEVICE)
        pred = model(batch, entity_table)
        pred = pred.view(-1) if pred.dim() > 1 and pred.size(1) == 1 else pred
        pred_list.append(pred.detach().cpu())
    return torch.cat(pred_list, dim=0).numpy()
# --- END MODIFY ---

In [13]:
# --- Orchestrator: run all discovered tasks in single & merged modes (robust) ---
import pandas as pd
from collections import defaultdict

MODES = globals().get("MODES", ["single", "merged"])

records = []
failures = []

for task_name in TASKS:
    for mode in MODES:
        try:
            res = run_tabpfn_on_task(DATASET, task_name, mode=mode)
            row = {
                "dataset": res.get("dataset", DATASET),
                "task": res.get("task", task_name),
                "mode": res.get("mode", mode),
                "n_train_used": res.get("n_train_used", None),
                "n_train_total": res.get("n_train_total", None),
                "n_val": res.get("n_val", None),
                "n_test": res.get("n_test", None),
            }
            # flatten metrics dicts if present
            for k, v in (res.get("val_metrics") or {}).items():
                row[f"val_{k}"] = v
            for k, v in (res.get("test_metrics") or {}).items():
                row[f"test_{k}"] = v
            records.append(row)
        except Exception as e:
            msg = f"[{DATASET} | {task_name} | {mode}] failed: {e!s}"
            print(msg)
            failures.append(msg)

results_df = pd.DataFrame.from_records(records)

if results_df.empty:
    print("No successful runs were recorded. Check the failure messages above.")
    # Create an empty, well-formed frame so downstream plotting code doesn't crash
    results_df = pd.DataFrame(
        columns=["dataset", "task", "mode", "n_train_used", "n_train_total", "n_val", "n_test"]
    )
else:
    # Ensure required sort keys exist even if some rows missed them
    for col in ["task", "mode"]:
        if col not in results_df.columns:
            results_df[col] = pd.NA
    # Sort only by the columns that exist to avoid KeyError
    sort_keys = [c for c in ["task", "mode"] if c in results_df.columns]
    if sort_keys:
        results_df = results_df.sort_values(sort_keys)

display(results_df)


/Users/michaelflppv/PycharmProjects/TabPFN/.venv1/lib/python3.10/site-packages/tabpfn/regressor.py:494: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(


[rel-f1 | driver-position | single] failed: got an unexpected keyword argument 'squared'
Loading Database object from /Users/michaelflppv/Library/Caches/relbench/rel-f1/db...
Done in 0.01 seconds.


/Users/michaelflppv/PycharmProjects/TabPFN/.venv1/lib/python3.10/site-packages/tabpfn/regressor.py:494: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(


[rel-f1 | driver-position | merged] failed: got an unexpected keyword argument 'squared'


/Users/michaelflppv/PycharmProjects/TabPFN/.venv1/lib/python3.10/site-packages/tabpfn/classifier.py:462: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(
/Users/michaelflppv/PycharmProjects/TabPFN/.venv1/lib/python3.10/site-packages/tabpfn/classifier.py:462: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(
/Users/michaelflppv/PycharmProjects/TabPFN/.venv1/lib/python3.10/site-packages/tabpfn/classifier.py:462: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(
/Users/michaelflppv/PycharmProjects/TabPFN/.venv1/lib/python3.10/site-packages/tabpfn/classifier.py:462: UserWarning: Running on CPU with more than 

,dataset,task,mode,n_train_used,n_train_total,n_val,n_test,val_average_precision,val_accuracy,val_f1,val_roc_auc,test_average_precision,test_accuracy,test_f1,test_roc_auc
1,rel-f1,driver-dnf,merged,1000,11411,566,702,0.762023,0.779152,0.875869,0.467338,0.707345,0.705128,0.827068,0.429278
0,rel-f1,driver-dnf,single,1000,11411,566,702,0.762023,0.779152,0.875869,0.467338,0.707345,0.705128,0.827068,0.429278
3,rel-f1,driver-top3,merged,1000,1353,588,726,0.262874,0.780612,0.145695,0.531705,0.206704,0.792011,0.038217,0.584161
2,rel-f1,driver-top3,single,1000,1353,588,726,0.262874,0.780612,0.145695,0.531705,0.206704,0.792011,0.038217,0.584161


In [14]:
# --- Persist results to CSV ---

out_dir = globals().get("OUT_DIR", "outputs")
os.makedirs(out_dir, exist_ok=True)

timestamp = time.strftime("%Y%m%d-%H%M%S")
csv_name = f"tabpfn_{DATASET}_{timestamp}.csv"
csv_path = os.path.join(out_dir, csv_name)

results_df.to_csv(csv_path, index=False)
print(f"Saved results to: {csv_path}")


Saved results to: outputs/tabpfn_rel-f1_20250815-153130.csv
